In [11]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('jugadores_ready.csv')

In [3]:
list_equip = list(df['equipo'].unique())

In [4]:
def Team(E):
    partidos = df.groupby('equipo').get_group(E)
    x = 1 #jornada a empezar
    lista = []
    for i in range(36): #numero de filas
        datos_a = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.jornada == i+x].iloc[0:12,0:7]))).T
        datos_a = datos_a.reset_index()
        corners_a = partidos[partidos.jornada == i+x].iloc[0:12,-1].sum()#suma de los corners
        datos_a.insert(6,'corners', corners_a)


        datos_b = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.jornada == i+x+1].iloc[0:12,0:7]))).T
        datos_b = datos_b.reset_index()
        corners_b = partidos[partidos.jornada == i+x+1].iloc[0:12,-1].sum()#suma de los corners
        datos_b.insert(6,'corners', corners_b)

        jugadores_c = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.jornada == i+x+2].iloc[0:12,0:1]))).T
        corners_c = partidos[partidos.jornada == i+x+2].iloc[0:12,-1].sum()
        jugadores_c.insert(11,'corners', corners_c)

        total_v1 = pd.concat([datos_a,datos_b,jugadores_c],axis=1,ignore_index=True)
        total_v1 = total_v1.drop([79,86,87,88,89,91],axis=1)
        lista.append(total_v1)

    df_final = pd.concat(lista)
    df_final = df_final.reset_index()

    df_final = df_final.drop(0,axis=1)
    df_final = df_final.drop('index',axis=1)
    df_final.replace(np.nan,0)
    # df_final = df_final.replace(['local','visitante'],[0,1])
    return df_final

## Random Forest

In [5]:
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.ensemble import RandomForestClassifier


from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [19]:
X = np.array(X)
y = np.array(y)
from sklearn.model_selection import GridSearchCV
grid_solver = GridSearchCV(estimator = clf, # model to train
                   param_grid = params, # param_grid
                   scoring = make_scorer(f1_score, average ="macro"),
                   cv = 5)
model_result = grid_solver.fit(X,y)

/home/angmar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/angmar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/angmar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/home/angmar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expec

In [20]:
model_result.best_estimator_

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=4, max_features=5,
                       max_leaf_nodes=8, max_samples=None,
                       min_impurity_decrease=0.02, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=2,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [22]:
model_result.best_params_

{'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 5,
 'max_leaf_nodes': 8,
 'min_impurity_decrease': 0.02,
 'min_samples_split': 3,
 'n_estimators': 2}

In [ ]:
y = np.array(y)

In [9]:
def RFTeam(X):
    #df_final=X    
    df_final = Team(X)
    df_final = df_final.replace(np.nan, 0.0)
    df_final[169] = df_final[169].replace((0,1,2,3),0)
    df_final[169] = df_final[169].replace((4,5,6),1)
    df_final[169] = df_final[169].replace((7,8,9,10,11,12,13),2)
    X = df_final.iloc[:,:162]
    y = df_final.iloc[:,162:163]
    X = np.array(X)
    y = np.array(y)
    #y = y.replace(np.nan,0.0)
    y = y[:, 0]
    # Build a forest and compute the feature importances
    forest = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=4, min_samples_split=3, max_features=2, max_leaf_nodes=8, min_impurity_decrease=0.02)
 #Change for R.Forest
    forest.fit(X, y)
    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    #print("Feature ranking:")

    #for f in range(X.shape[1]):
      #  print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    X = df_final.iloc[:,indices[:50]]
    X=X.replace(np.nan,0)
    X = np.array(X)

    loo = LeaveOneOut()
    loo.get_n_splits(X)
    yhat=[]
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        #CLASIFICADOR Random Forest
        clf = RandomForestClassifier(n_estimators=100, criterion="entropy", max_depth=4, min_samples_split=3, max_features=2, max_leaf_nodes=8, min_impurity_decrease=0.02)
        clf.fit(X_train, y_train)  ##X_train, y_train
        yhat1 = clf.predict(X_test)
        yhat.append(yhat1)       
        
    #print("Accuracy: ", accuracy_score(y,yhat))
    return accuracy_score(y,yhat)

In [12]:
RFTeam('betis')

0.5277777777777778

In [13]:
L_Acc = []
for i in list_equip:
    L_Acc.append(RFTeam(i))
Dicc = dict(zip(list_equip,L_Acc))
Dicc

{'girona': 0.6666666666666666,
 'valladolid': 0.3611111111111111,
 'betis': 0.4722222222222222,
 'levante': 0.3888888888888889,
 'celta': 0.6666666666666666,
 'espanyol': 0.6388888888888888,
 'villarreal': 0.5,
 'real_sociedad': 0.4722222222222222,
 'barcelona': 0.4444444444444444,
 'alaves': 0.6111111111111112,
 'eibar': 0.3888888888888889,
 'huesca': 0.5555555555555556,
 'rayo': 0.7222222222222222,
 'sevilla': 0.5555555555555556,
 'real_madrid': 0.4166666666666667,
 'getafe': 0.4166666666666667,
 'valencia': 0.4444444444444444,
 'atletico': 0.3333333333333333,
 'athletic': 0.6944444444444444,
 'leganes': 0.6111111111111112}

In [14]:
#Media Adaboost: 0.4375
Media_Random_Forest = sum(L_Acc)/20
Media_Random_Forest

0.5180555555555555